In [45]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt

In [96]:
# Retreiving active rosters

# Instantiating necessary items
tables = pd.read_html('https://www.lineups.com/nba/depth-charts')
teams = ['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets',
        'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets',
        'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
        'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat',
        'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans',
        'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers',
        'Phoenix Suns', 'Portland Trailblazers', 'Sacramento Kings', 'San Antonio Spurs',
        'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
def name_adjustment(x):
    try:
        names = x.split(' ')
        if len(names) == 4:
            name = names[0] + ' ' + names[1]
        if len(names) == 6:
            name = names[0] + ' ' + names[1] + ' ' + names[2]
    except:
        name = x
    return name

# Getting active rosters into dictionary of lists for each team
team_dict = {}
for team, table in zip(teams,tables):
    table.columns = table.columns.droplevel(0)
    for i in [1,2,3]:
        table[str(i)] = table[str(i)].apply(name_adjustment)
    table = table[['1', '2', '3']]
    players = list()
    for i in range(table.shape[0]): 
        for j in range(table.shape[1]):
            player = table.iloc[i, j]
            players = players + [player]
    team_dict[team] = players

In [97]:
# Retreiving games played table

# Map to change team names
games_played_map = {
    'Milwaukee' : 'Milwaukee Bucks',
    'Boston' : 'Boston Celtics',
    'Memphis' : 'Memphis Grizzlies',
    'LA Clippers' : 'Los Angeles Clippers',
    'Sacramento' : 'Sacramento Kings',
    'Indiana' : 'Indiana Pacers',
    'Golden State' : 'Golden State Warriors',
    'LA Lakers' : 'Los Angeles Lakers',
    'San Antonio' : 'San Antonio Spurs',
    'Utah' : 'Utah Jazz',
    'Houston' : 'Houston Rockets',
    'Dallas' : 'Dallas Mavericks',
    'Charlotte' : 'Charlotte Hornets',
    'Cleveland' : 'Cleveland Cavaliers',
    'Orlando' : 'Orlando Magic',
    'Washington' : 'Washington Wizards',
    'New York' : 'New York Knicks',
    'Brooklyn' : 'Brooklyn Nets',
    'Philadelphia' : 'Philadelphia 76ers',
    'Minnesota' : 'Minnesota Timberwolves',
    'Miami' : 'Miami Heat',
    'New Orleans' : 'New Orleans Pelicans',
    'Detroit' : 'Detroit Pistons',
    'Okla City' : 'Oklahoma City Thunder',
    'Portland' : 'Portland Trailblazers',
    'Chicago' : 'Chicago Bulls',
    'Denver' : 'Denver Nuggets',
    'Phoenix' : 'Phoenix Suns',
    'Atlanta' : 'Atlanta Hawks',
    'Toronto' : 'Toronto Raptors'
}

# Reading in table and adjusting columns
tables = pd.read_html('https://www.teamrankings.com/nba/stat/games-played')
games_played_table = tables[0]
games_played_table = games_played_table[['Team', str(current_year - 1)]]
games_played_table.columns = ['Team', 'Games_Played']
games_played_table['Team'] = games_played_table.Team.apply(lambda x: games_played_map[x])

frac_season = np.mean(games_played_table.Games_Played)/82

/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [98]:
# Retreiving BOY projected VORPS
boy_vorps = pd.read_csv('In_Season/Data/opening_day_vorps_player.csv', index_col = 0)
# Changing Vorp_Projection column to be a float
boy_vorps['VORP_projection'] = boy_vorps.VORP_projection.apply(lambda x: float(x.strip('[]')) if type(x)==str else x)

In [99]:
current_year = 2022

# Retrieving current year VORPs
tables = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{str(current_year)}_advanced.html')
table = tables[0]
table = table[['Player', 'Tm', 'G', 'VORP']]
table.columns = ['Player', 'Team', 'Games', 'VORP']
table = table[table.Team != 'Tm']
table = table[table.Team != 'TOT']
table['VORP'] = table.VORP.apply(pd.to_numeric)
player_vorp = table.groupby(['Player', 'Games'])['VORP'].sum()
player_vorp = pd.DataFrame(player_vorp)
player_vorp.reset_index(drop = False, inplace = True)
player_vorp.columns = ['Player', 'Games', 'VORP']

# Changing Player column to align with naming conventions in BOY_vorps
# def name_exceptions(x):
#     if x == 'cam thomas':
#         return 'cameron thomas'
#     if x == 'herbert jones':
#         return 'herb jones'
#     if x == 'charlie brown':
#         return 'charles brown'
#     if x == 'ish wainright':
#         return 'ishmail wainright'
#     if x == 'enes freedom':
#         return 'enes kanter'

#     return x

table['Player'] = table.Player.str.lower()
table['Player'] = table.Player.apply(unidecode.unidecode)
table['Player'] = table.Player.str.replace("'", '')
table['Player'] = table.Player.str.replace(".", '')
table['Player'] = table.Player.apply(lambda x: x.split(' ')[0] + ' ' + x.split(' ')[1])
table['Player'] = table.Player.str.strip('*,')
table['Player'] = table.Player.apply(name_exceptions)



In [100]:
vorp_df = pd.merge(boy_vorps, table, on = 'Player', how = 'inner')
vorp_df['Games'] = vorp_df.Games.apply(float)
vorp_df.drop_duplicates(subset = ['Player'], inplace = True)

In [101]:
# Getting each player's annualized VORP, adjusting for games played
vorp_df['VORP_82'] = 0
if frac_season < 0.25:
    vorp_df['VORP_82'] = vorp_df.VORP_projection
if frac_season > 0.25:
    for index, row in vorp_df.iterrows():
        if ((frac_season <= 0.5) & (row.Games < 12)):
            vorp_df.loc[index, 'VORP_82'] = row.VORP_projection
        elif ((frac_season > 0.5) & (frac_season <= 0.75) & (row.Games < 25)):
            vorp_df.loc[index, 'VORP_82'] = row.VORP_projection
        elif ((frac_season > 0.75) & (row.Games < 40)):
            vorp_df.loc[index, 'VORP_82'] = row.VORP_projection
        else:
            games_frac = float(row.Games)/82.0
            vorp_df.loc[index, 'VORP_82'] = (row.VORP_projection * (1.0 - games_frac)) + (row.VORP * games_frac)

In [102]:
active_rosters = team_dict

In [103]:
team_vorp_df = pd.DataFrame(columns = ['Team', 'VORP_Today'])
missed_players = list()
for team, roster in active_rosters.items():
    team_vorp = 0
    for player in roster:
        if str(player) == 'nan':
            continue
        
        # Aligning naming conventions with vorp_df
        player = player.lower()
        player = unidecode.unidecode(player)
        player = player.replace("'", '')
        player = player.replace(".", '')
        player = player.split(' ')[0] + ' ' + player.split(' ')[1]
        player.strip('*,')
        player = name_exceptions(player)
        
        # Adding player's VORP to team VORP total
        player_vorp = vorp_df[vorp_df.Player == player]
        if len(player_vorp == 1):
            vorp = sum(player_vorp.VORP_82)
            team_vorp += vorp
        else:
            missed_players.append(player)
    
    # Adding team's VORP to overall VORP df
    series = pd.Series([team, team_vorp], index = team_vorp_df.columns)
    team_vorp_df = team_vorp_df.append(series, ignore_index = True)

In [105]:
active_rosters

{'Atlanta Hawks': ['Trae Young',
  'Delon Wright',
  'Lou Williams',
  'Bogdan Bogdanovic',
  'Timothe Luwawu-Cabarrot',
  'Skylar Mays',
  'Kevin Huerter',
  'Cam Reddish',
  'Jalen Johnson',
  'John Collins',
  'Danilo Gallinari',
  nan,
  'Clint Capela',
  'Onyeka Okongwu',
  'Gorgui Dieng'],
 'Boston Celtics': ['Marcus Smart',
  'Payton Pritchard',
  'Romeo Langford',
  'Jaylen Brown',
  'Dennis Schroder',
  'Brodric Thomas',
  'Jayson Tatum',
  'Josh Richardson',
  'Aaron Nesmith',
  'Robert Williams III',
  'Grant Williams',
  'Sam Hauser',
  'Al Horford',
  'Enes Freedom',
  'Bruno Fernando'],
 'Brooklyn Nets': ['Kyrie Irving',
  'Patty Mills',
  'Jevon Carter',
  'James Harden',
  'David Duke Jr.',
  'Cam Thomas',
  'Kevin Durant',
  'Bruce Brown',
  'Kessler Edwards',
  "DeAndre' Bembry",
  'James Johnson',
  'Blake Griffin',
  'LaMarcus Aldridge',
  'Nicolas Claxton',
  "Day'Ron Sharpe"],
 'Charlotte Hornets': ['LaMelo Ball',
  'Ish Smith',
  'James Bouknight',
  'Terry Rozie